In [ ]:
import pandas as pd
import struct
import datetime
from plotly.subplots import make_subplots
import numpy as np
import plotly.graph_objects as go
from dotenv import load_dotenv
import os

In [ ]:
QUERY = """
select 
    lower(hex(bytes)) as packets_hex,
    bytes as packet
from packets 
where (lower(hex(bytes)) like "aa6400a1%" or lower(hex(bytes)) like "aa5c00f0%")and lower(hex(uuid)) = "610800058d6d82b8614a1c8cb0f8dcc6"
"""

In [ ]:
DATABASE_URL = os.getenv("DATABASE_URL").replace("sqlite://", "sqlite:///../")
df = pd.read_sql(QUERY, DATABASE_URL)

In [ ]:
df["datetime"] = pd.to_datetime(df["packet"].apply(lambda data: struct.unpack('<I', data[11:15])[0]), unit="s")
df = df.sort_values("datetime", ascending=True)

df['date'] = df['datetime'].dt.date
df['time'] = df['datetime'].dt.time
df["bpm"] = df["packet"].apply(lambda data: data[21])

In [ ]:
df["sleep_stage"] = df["packet"].apply(lambda data: struct.unpack('<I', data[31:35])[0])

In [ ]:
def plot_heart_rate(days):
    for day_data in days:
        # Create a figure with secondary y-axes
        fig = make_subplots(specs=[[{"secondary_y": True}]])

        # Add heart rate trace to primary y-axis
        fig.add_trace(
            go.Scatter(x=day_data['datetime'], y=day_data['bpm'], mode='lines', name='Heart Rate (BPM)', line=dict(color='blue')),
            secondary_y=False,
        )

        # Add x trace to secondary y-axis
        fig.add_trace(
            go.Scatter(x=day_data['datetime'], y=day_data['sleep_stage'], mode='lines', name='X', line=dict(color='orange')),
            secondary_y=True,
        )

        # Update layout for titles and axes
        fig.update_layout(
            title=f"Heart Rate and XYZ from {day_data['datetime'].iloc[0].date()} Noon to Next Day Noon",
            xaxis_title="Time",
            legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        )

        # Update x-axis for time formatting
        fig.update_xaxes(tickformat='%H:%M')

        # Update y-axes titles
        fig.update_yaxes(title_text="Heart Rate (BPM)", secondary_y=False)
        fig.update_yaxes(title_text="XYZ Values", secondary_y=True)

        # Show the plot
        fig.show()

In [ ]:
def filter_noon_to_noon(df):
    # Group the data by date
    days = []
    unique_dates = df['date'].unique()
    
    for date in unique_dates:
        # Define noon of the current day and noon of the next day
        start_noon = pd.Timestamp(datetime.datetime.combine(date, datetime.time(12, 0)))
        end_noon = start_noon + pd.Timedelta(days=1)
        
        # Filter data between the start and end noon
        day_data = df[(df['datetime'] >= start_noon) & (df['datetime'] < end_noon)].copy()
        if not day_data.empty:
            days.append(day_data)


    return days

In [ ]:
days = filter_noon_to_noon(df)
plot_heart_rate(days)